In [1]:
abstract type AbstractNode end

In [2]:
abstract type LeafNode <: AbstractNode end

In [3]:
mutable struct Variable{T} <: LeafNode
    value::T
    grad::T

    Variable(val::T) where T = new{T}(val)
    Variable(val::T, grad::T) where T = new{T}(val)
end

In [4]:
struct Node{FT <: Function, ArgsT <: Tuple, KwargsT <: NamedTuple} <: AbstractNode
    f::FT
    args::ArgsT
    kwargs::KwargsT
end

In [ ]:
import Pkg; Pkg.add("YAAD")

In [16]:
abstract type Operator end

module Trait
import YAAD: Operator

struct Method{FT} <: Operator
    f::FT
end

struct Broadcasted{FT} <: Operator
    f::FT
end
end # Trait

Main.Trait

In [17]:
struct Node{FT <: Operator, ArgsT <: Tuple, KwargsT <: NamedTuple} <: AbstractNode
    f::FT
    args::ArgsT
    kwargs::KwargsT
end

LoadError: [91minvalid redefinition of constant Node[39m

In [18]:
# wrap function to Method
Node(f::Function, args, kwargs) = Node(Trait.Method(f), args, kwargs)
Node(op, args) = Node(op, args, NamedTuple())

Node

In [19]:
mutable struct CachedNode{NT <: AbstractNode, OutT} <: AbstractNode
    node::NT
    output::OutT
end

In [20]:
function CachedNode(f, args...; kwargs...)
    node = Node(f, args, kwargs.data) # this constructs a Node
    output = forward(node)
    CachedNode(node, output)
end

CachedNode

In [21]:
forward(node::Node) = forward(node.f, map(forward, node.args)...; map(forward, node.kwargs)...)

forward (generic function with 1 method)

In [22]:
struct Linear <: Operator
  w::Matrix{Float64}
  b::Vector{Float64}
end

forward(op::Linear, x::Vector{Float64}) = op.w * x + b

forward (generic function with 2 methods)

In [23]:
forward(node::CachedNode) = (node.output = forward(node.node))

forward (generic function with 3 methods)

In [25]:
function forward(::Method{typeof(sin)}, x)
  sin(x)
end

LoadError: [91mTypeError: in Type{...} expression, expected UnionAll, got Type{Method}[39m

In [26]:
forward(op::Operator, args...; kwargs...) = op.f(args...; kwargs...)

forward (generic function with 4 methods)

In [27]:
(op::Linear)(x::Vector) = op.w * x + b